In [1]:
import os
import cv2
import numpy as np


In [25]:
!pip uninstall tensorflow

^C


In [26]:
!pip install tensorflow-gpu

^C


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "C:\Users\wwwth\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\wwwth\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\wwwth\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
    

In [27]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

[]

In [4]:
def load_dataset(image_folder, coordinates_folder):
    images = []
    coordinates = []
    
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"):
            # Load image
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)
            images.append(image)
            
            # Load coordinates
            coordinates_path = os.path.join(coordinates_folder, "coordinates_"+ filename[:-4] + ".txt")
            with open(coordinates_path, 'r') as f:
                coords = [float(coord) for coord in f.readline().split()]
                coordinates.append(coords)
                
    return np.array(images), np.array(coordinates)
image_folder = 'frames_folder'
coordinates_folder = 'coordinates'
images, coordinates = load_dataset(image_folder, coordinates_folder)


In [11]:
normalized_images = np.array(images) / 255.0

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2)  # Output layer with 2 units for x and y coordinates
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Assuming coordinates are continuous values
              metrics=['mse'])  # Mean Squared Error

# Train the model
history = model.fit(normalized_images, coordinates, epochs=10, validation_split=0.2)

# Evaluate the model
test_loss, test_mse = model.evaluate(test_images, test_coordinates)

# Optionally, save the model
model.save('gaze_point_model.h5')


c:\Users\wwwth\anaconda3\envs\ultra\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'x_train' is not defined